In [ ]:
%pipinstallazure-cognitiveservices-speech==1.37.0

In [ ]:
#key_vault_namevalueissetatthetimeofdeployment

key_vault_name='kv_to-be-replaced'

In [ ]:
fromtrident_token_library_wrapperimportPyTridentTokenLibraryastl

defget_secrets_from_kv(kv_name,secret_name):

access_token=mssparkutils.credentials.getToken("keyvault")
kv_endpoint=f'https://{kv_name}.vault.azure.net/'
return(tl.get_secret_with_token(kv_endpoint,secret_name,access_token))

openai_api_type="azure"
openai_api_version=get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-VERSION")
openai_api_base=get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-ENDPOINT")
openai_api_key=get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-KEY")

In [ ]:
#SetAIservicesvariables
ai_services_endpoint=get_secrets_from_kv(key_vault_name,"COG-SERVICES-ENDPOINT")
ai_services_key=get_secrets_from_kv(key_vault_name,"COG-SERVICES-KEY")
ai_services_region=get_secrets_from_kv(key_vault_name,"COG-SERVICES-REGION")
#wav_file_path='/lakehouse/default/Files/data/audio_input/Travel_20240417132839.wav'
language1='en-US'

In [ ]:
##Thiscellcreatesnewfolderswithinthespecifiedbasepathinthelakehouse.
#Thepurposeistocreatecorrespondingfolderssofilescanbemovedastheyareprocessed.
importos

#Definethebasepath
base_path='/lakehouse/default/Files/data'

#Listoffolderstobecreated
folders=['audio_failed','audio_processed']

#Createeachfolder
forfolderinfolders:
folder_path=os.path.join(base_path,folder)
try:
os.makedirs(folder_path,exist_ok=True)
print(f'Foldercreatedat:{folder_path}')
exceptExceptionase:
print(f'Failedtocreatethefolder{folder_path}.Error:{e}')

In [ ]:
#Dropthemetadatatableifitalreadyexists
spark.sql('droptableifexistsckm_conv_metadata')

In [ ]:
frompyspark.sqlimportfunctionsasF

spark.sql("setspark.sql.legacy.timeParserPolicy=LEGACY")

#ReadalltheCSVfilesinthedirectory
df=spark.read.format("csv").option("header","true").load("Files/data/audio_input/*.csv")

#ConvertStartTimeandEndTimetotimestampformat
df=df.withColumn("StartTime",F.to_timestamp("StartTime","MM/dd/yyyyh:mm:ssa"))
df=df.withColumn("EndTime",F.to_timestamp("EndTime","MM/dd/yyyyh:mm:ssa"))

#Calculatethedurationinmillisecondsandadditasanewcolumn
df=df.withColumn("Duration",(F.col("EndTime").cast("long")-F.col("StartTime").cast("long"))/60)


#WritetheDataFrametoaDeltatable
df.write.format('delta').mode('overwrite').saveAsTable('ckm_conv_metadata')

##Displaythefirst2rows
#display(df.head(2))


In [ ]:
#ThiscodeblockisdesignedtotranscribespeechfromanaudiofileusingAzure'sCognitiveServicesSpeechSDK.
#https://learn.microsoft.com/en-us/azure/ai-services/speech-service/get-started-stt-diarization?tabs=windows&pivots=programming-language-python

#Itsupportsdiarization,whichdistinguishesbetweendifferentspeakersintheaudio.
#Thetranscribedresultsarereturnedasalistofallrecognizedutteranceswithassociatedmetadata.

importos
importtime
importazure.cognitiveservices.speechasspeechsdk
importjson

#Functiontotranscribespeechfromanaudiofile
deftranscribe_from_file(ai_services_key,ai_services_region,wav_file_path,conversation_id):
#Listtostoretheresultsofthetranscription
all_results=list()

#Configurethespeechservice
speech_config=speechsdk.SpeechConfig(subscription=ai_services_key,region=ai_services_region)
speech_config.speech_recognition_language="en-US"

#Setuptheaudioconfigurationusingtheprovidedfilepath
audio_config=speechsdk.audio.AudioConfig(filename=wav_file_path)
#Createaconversationtranscriberobject
conversation_transcriber=speechsdk.transcription.ConversationTranscriber(speech_config=speech_config,audio_config=audio_config)

#Flagtoindicatewhentostoptranscribing
transcribing_stop=False

#Callbackforwhenthetranscriptionsessionstarts
defconversation_transcriber_session_started_cb(evt:speechsdk.SessionEventArgs):
print('SessionStartedevent')

#Callbacktosignaltostopcontinuousrecognition
defstop_cb(evt:speechsdk.SessionEventArgs):
nonlocaltranscribing_stop
transcribing_stop=True
#LogthesessionID
print(f"Stoppingtranscriptionforsessionid:{evt.session_id}")

#Checkiftheeventhasaresultattribute
ifhasattr(evt,'result'):
#Iftheresultreasoniscancellation,providethecancellationdetails
ifevt.result.reason==speechsdk.ResultReason.Canceled:
cancellation_details=speechsdk.CancellationDetails(evt.result)
print(f"Transcriptionwasstoppedduetocancellation:{cancellation_details.reason}")
ifcancellation_details.reason==speechsdk.CancellationReason.Error:
print(f"Errordetails:{cancellation_details.error_details}")
#IftheresultreasonisEndOfStream,indicatetheaudiostreamhasended
elifevt.result.reason==speechsdk.ResultReason.EndOfStream:
print("Transcriptionstoppedbecausetheendoftheaudiostreamwasreached.")
#IftheresultreasonisNoMatch,indicatenospeechcouldberecognized
elifevt.result.reason==speechsdk.ResultReason.NoMatch:
print("Transcriptionstoppedbecausenospeechcouldberecognized.")
#Foranyotherreason,logtheresultreason
else:
print(f"Transcriptionstoppedforanunknownreason:{evt.result.reason}")
else:
#Ifthereisnoresultattribute,logthatthereasonisunknown
print("Transcriptionstopped,butnoadditionalinformationisavailable.")

#Callbackforwhenthetranscriptioniscanceled
defconversation_transcriber_recognition_canceled_cb(evt:speechsdk.SessionEventArgs):
print("Canceledevent")
#Accessthecancellationdetailsfromtheevent
cancellation_details=speechsdk.CancellationDetails(evt.result)
#Printthereasonforthecancellation
print(f"Canceledevent:{cancellation_details.reason}")

#Iftherewasanerror,printtheerrordetails
ifcancellation_details.reason==speechsdk.CancellationReason.Error:
print(f"Errordetails:{cancellation_details.error_details}")


#Callbackforwhenthetranscriptionsessionstops
defconversation_transcriber_session_stopped_cb(evt:speechsdk.SessionEventArgs):
#Printthesessionstoppedeventwiththesessionidforreference
print(f"SessionStoppedeventforsessionid:{evt.session_id}")

#Iftheeventhasaresultattribute,wecancheckifthereareanyadditionaldetails
ifhasattr(evt,'result')andevt.result:
#Checkiftheresulthasareasonattributeandprintit
ifhasattr(evt.result,'reason'):
print(f"Reasonforstop:{evt.result.reason}")

#Iftheresultisacancellation,printthecancellationdetails
ifevt.result.reason==speechsdk.ResultReason.Canceled:
cancellation_details=speechsdk.CancellationDetails(evt.result)
print(f"Cancellationreason:{cancellation_details.reason}")
ifcancellation_details.reason==speechsdk.CancellationReason.Error:
print(f"Errordetails:{cancellation_details.error_details}")


#Handlerforthefinalresultofthetranscription
defhandle_final_result(evt):
nonlocalall_results
#Logtheeventtype
print(f"Eventtype:{type(evt)}")
#Logtheevent'sresultreason
print(f"Resultreason:{evt.result.reason}")

#Checkiftheevent'sresultisspeechrecognitionwitharecognizedphrase
ifevt.result.reason==speechsdk.ResultReason.RecognizedSpeech:
#ParsetheJSONresultfromthetranscription
r=json.loads(evt.result.json)
#LogtheentireJSONresult
print(f"JSONresult:{r}")
#Appendtherelevantdatatotheresultslist
all_results.append([conversation_id,
r["Id"],
r["DisplayText"],
r["Offset"],
r["Duration"],
r["Channel"],
r["Type"],
r["SpeakerId"]
])
#Iftheresultreasonisnotrecognizedspeech,logthatnorecognizedspeechwasfound
else:
print("Norecognizedspeechwasfound.")


#Connectthecallbackstotheeventsfiredbytheconversationtranscriber
conversation_transcriber.transcribed.connect(handle_final_result)
conversation_transcriber.session_started.connect(conversation_transcriber_session_started_cb)
conversation_transcriber.session_stopped.connect(conversation_transcriber_session_stopped_cb)
conversation_transcriber.canceled.connect(conversation_transcriber_recognition_canceled_cb)
conversation_transcriber.session_stopped.connect(stop_cb)
conversation_transcriber.canceled.connect(stop_cb)

#Starttheasynchronoustranscription
conversation_transcriber.start_transcribing_async()

#Waitforthetranscriptiontocomplete
whilenottranscribing_stop:
time.sleep(.5)

#Stoptheasynchronoustranscription
conversation_transcriber.stop_transcribing_async()
#Returnthelistoftranscribedresults
return(all_results)


In [ ]:
##spark.sql('droptableifexistsckm_conv_messages')

#frompyspark.sqlimportSparkSession

##CreateaSparksession
#spark=SparkSession.builder.getOrCreate()

##Gettheschemaoftheexistingtable
#schema=spark.table("ckm_conv_messages").schema

##CreateanemptyDataFramewiththesameschema
#empty_df=spark.createDataFrame([],schema)

##OverwritetheexistingtablewiththeemptyDataFrame
#empty_df.write.mode('overwrite').saveAsTable("ckm_conv_messages")


In [ ]:
spark.sql('droptableifexistsckm_conv_messages')

In [ ]:
"""
ThisscripttranscribesaudiofilesusingtheAIserviceskeyandregion.Ititeratesovereachrowinadataframe,
constructsthefullpathoftheaudiofile,andattemptstotranscribetheaudio.Ifthetranscriptionissuccessful
andnotempty,itcreatesanewdataframewiththetranscriptionsandwritesittoadeltatable'ckm_conv_messages'.
The'ckm_conv_messages'tablestorestheconversationmessageswithcolumnssuchasconversation_id,Id,DisplayText,
Offset,Duration,Channel,Type,andSpeakerId.Theprocessedaudiofilesarethenmovedtoa'audio_processed'folder.
Ifanerroroccursduringtheprocess,itprintstheerrormessageandthefilethatcouldnotbeloaded.
"""

frompyspark.sqlimportfunctionsasf

forrowindf.rdd.collect():
#Stripleadingandtrailingwhitespacefromthefilename
file_name=row.FileName.strip()
wav_file_path='/lakehouse/default/Files/data/audio_input/'+file_name#fullpathisrequiredforspeechSDK
#print(wav_file_path)
try:
#print(f"transcribingfile:{wav_file_path}")
r=transcribe_from_file(ai_services_key,ai_services_region,wav_file_path,row.ConversationId)
#print(f"r={r}")
iflen(r)!=0:
df_columns=["conversation_id","Id","DisplayText","Offset","Duration","Channel","Type","SpeakerId"]
df_conv=spark.createDataFrame(data=r,schema=df_columns)
df_conv=df_conv.coalesce(1).withColumn("row_id",f.monotonically_increasing_id())

df_conv.write.format('delta').mode('append').saveAsTable('ckm_conv_messages')
#Movetheprocessedfiletothe'audio_processed'folder
mssparkutils.fs.mv(('Files/data/audio_input/'+file_name),('Files/data/audio_processed/'+file_name),False,True)
#break
exceptExceptionase:
print("couldnotload:",wav_file_path)
print("Anerroroccurred:",e)#Printtheexception
#Movetheprocessedfiletothe'audio_failed'folder
mssparkutils.fs.mv(('Files/data/audio_input/'+file_name),('Files/data/audio_failed/'+file_name),False,True)


In [ ]:
importos
importshutil

#Directorypaths
input_dir='/lakehouse/default/Files/data/audio_input/'
processed_dir='/lakehouse/default/Files/data/audio_processed/'

#Getalistofall.csvfilesintheinputdirectory
csv_files=[fforfinos.listdir(input_dir)iff.endswith('.csv')]

#Moveeach.csvfiletotheprocesseddirectory
forfile_nameincsv_files:
shutil.move(os.path.join(input_dir,file_name),os.path.join(processed_dir,file_name))
